input args:  Namespace(f='C:\\Users\\Ray Jang\\AppData\\Roaming\\jupyter\\runtime\\kernel-a938bf43-203e-4e0e-b155-40fc05dddeb1.json', graph_def=False, input_fld='.', input_model_file='mobilenet_convs.h5', num_outputs=1, output_fld='.', output_graphdef_file='model.ascii', output_model_file='mobilenet_CONV_cifar100.pb', output_node_prefix='output_node')


In [ ]:
# 출처: https://github.com/amir-abdi/keras_to_tensorflow/blob/master/keras_to_tensorflow.ipynb
# setting input arguments
import argparse
parser = argparse.ArgumentParser(description='set input arguments')
parser.add_argument('-input_fld', action="store", 
                    dest='input_fld', type=str, default='.')

parser.add_argument('-output_fld', action="store", 
                    dest='output_fld', type=str, default='.')

parser.add_argument('-input_model_file', action="store", 
                    dest='input_model_file', type=str, default='mobilenet_convs.h5')

parser.add_argument('-output_model_file', action="store", 
                    dest='output_model_file', type=str, default='mobilenet_CONV_cifar100.pb')

parser.add_argument('-output_graphdef_file', action="store", 
                    dest='output_graphdef_file', type=str, default='model.ascii')

parser.add_argument('-num_outputs', action="store", 
                    dest='num_outputs', type=int, default=1)

parser.add_argument('-graph_def', action="store", 
                    dest='graph_def', type=bool, default=False)

parser.add_argument('-output_node_prefix', action="store", 
                    dest='output_node_prefix', type=str, default='output_node')

parser.add_argument('-f')
args = parser.parse_args()
print('input args: ', args)

In [2]:
num_output = args.num_outputs

In [3]:
from keras.models import load_model
import tensorflow as tf
import os
import os.path as osp
from keras import backend as K

output_fld =  args.output_fld
if not os.path.isdir(output_fld):
    os.mkdir(output_fld)
weight_file_path = osp.join(args.input_fld, args.input_model_file)

Using TensorFlow backend.


In [4]:
K.set_learning_phase(0)
net_model = load_model(weight_file_path)

pred = [None]*num_output
pred_node_names = [None]*num_output
for i in range(num_output):
    pred_node_names[i] = args.output_node_prefix+str(i)
    pred[i] = tf.identity(net_model.outputs[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)

output nodes names are:  ['output_node0']


C:\Users\Ray Jang\Anaconda2\envs\venv\lib\site-packages\keras\models.py:251: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
sess = K.get_session()

if args.graph_def:
    f = args.output_graphdef_file 
    tf.train.write_graph(sess.graph.as_graph_def(), output_fld, f, as_text=True)
    print('saved the graph definition in ascii format at: ', osp.join(output_fld, f))

In [6]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
graph_io.write_graph(constant_graph, output_fld, args.output_model_file, as_text=False)
print('saved the freezed graph (ready for inference) at: ', osp.join(output_fld, args.output_model_file))

INFO:tensorflow:Froze 135 variables.
Converted 135 variables to const ops.
saved the freezed graph (ready for inference) at:  .\mobilenet_CONV_cifar100.pb
